# 语义内核

在这个代码示例中，您将使用 [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI框架来创建一个基础代理。

本示例的目标是向您展示我们在后续代码示例中实现不同代理模式时将使用的步骤。


## 导入所需的 Python 包


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## 创建客户端

在本示例中，我们将使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 来访问 LLM。

`ai_model_id` 被定义为 `gpt-4o-mini`。尝试将模型更改为 GitHub Models 市场中提供的其他模型，以查看不同的结果。

为了使用 `Azure Inference SDK`（用于 GitHub Models 的 `base_url`），我们将在 Semantic Kernel 中使用 `OpenAIChatCompletion` 连接器。此外，还有其他 [可用连接器](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)，可以将 Semantic Kernel 用于其他模型提供商。


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## 创建代理

下面我们创建名为 `TravelAgent` 的代理。

在这个示例中，我们使用了非常简单的指令。您可以更改这些指令，观察代理如何做出不同的响应。


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## 运行代理

现在我们可以通过定义一个类型为 `ChatHistoryAgentThread` 的线程来运行代理。任何必要的系统消息都可以通过 `invoke_stream` 的 `messages` 关键字参数提供给代理。

定义完这些后，我们创建一个 `user_inputs`，它代表用户发送给代理的内容。在这个例子中，我们将消息设置为 `Plan me a sunny vacation`。

您可以随意更改此消息，看看代理会有怎样不同的回应。


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
